In [385]:
import os
import h5py
import numpy as np
from IPython.lib.deepreload import reload
import matplotlib.pyplot as plt
from collections import OrderedDict
import skimage.io as io
from tqdm import tqdm
import time as ttime

%matplotlib qt

In [386]:
import xrdmaptools
from xrdmaptools.XRDMap import XRDMap
from xrdmaptools.ImageMap import ImageMap
from xrdmaptools.reflections.SpotModels import GaussianFunctions
reload(xrdmaptools);

In [ ]:
from xrdmaptools.utilities.math import tth_2_d, d_2_tth, energy_2_wavelength, tth_2_q, q_2_tth
from xrdmaptools.reflections.SpotModels import GaussianFunctions, LorentzianFunctions
from itertools import permutations
from itertools import product



# Re-write for the crystal class...
def metric_tensor(a, b, c, alpha, beta, gamma):
    A = a**2
    B = b**2
    C = c**2
    D = 2 * b * c * np.cos(alpha)
    E = 2 * a * c * np.cos(beta)
    F = 2 * a * b * np.cos(gamma)
    return (A, B, C, D, E, F)

def S_tensor(a, b, c, alpha, beta, gamma):
    V = a * b * c * np.sqrt(1
                            - np.cos(alpha)**2
                            - np.cos(beta)**2
                            - np.cos(gamma)**2
                            - 2 * np.cos(alpha) * np.cos(beta) * np.cos(gamma))
    S11 = b**2 * c**2 * np.sin(alpha)**2
    S22 = a**2 * c**2 * np.sin(beta)**2
    S33 = a**2 * b**2 * np.sin(gamma)**2
    S12 = a * b * c**2 * (np.cos(alpha) * np.cos(beta) - np.cos(gamma))
    S23 = a**2 * b * c * (np.cos(beta) * np.cos(gamma) - np.cos(alpha))
    S13 = a * b**2 * c * (np.cos(gamma) * np.cos(alpha) - np.cos(beta))

    return (V, S11, S22, S33, S12, S23, S13)


def hkl_2_d(hkls, a, b, c, alpha, beta, gamma):
    VS = np.round(S_tensor(a, b, c, alpha, beta, gamma), 8)

    d_list = []
    for hkl in hkls:
        h, k, l = hkl
        d = VS[0] * (VS[1] * (h**2)
                     + VS[2] * (k**2)
                     + VS[3] * (l**2)
                     + VS[4] * (h * k) * 2
                     + VS[5] * (k * l) * 2
                     + VS[6] * (h * l * 2)
                     )**(-0.5)
        
        d_list.append(d)
    
    return np.array(d_list)


def d_spacings(q_range, a, b, c, alpha, beta, gamma):
    q_min, q_max = q_range
    
    possible_hkls = list(product(range(10), repeat=3))[1:]

    VS = np.round(S_tensor(a, b, c, alpha, beta, gamma), 8)

    d_list = []
    for hkl in possible_hkls:
        h, k, l = hkl
        d = VS[0] * (VS[1] * (h**2)
                     + VS[2] * (k**2)
                     + VS[3] * (l**2)
                     + VS[4] * (h * k) * 2
                     + VS[5] * (k * l) * 2
                     + VS[6] * (h * l * 2)
                     )**(-0.5)
        
        if 2 * np.pi / d < q_min:
            continue
        elif 2 * np.pi / d > q_max:
            continue
        else:
            d_list.append(d)
    
    d_list = np.unique(np.round(d_list, 8))
        
    return sorted(d_list, reverse=True)


def pattern_estimate_guess(tth, intensity, a, b, c, alpha, beta, gamma, wavelength=test.wavelength, window=20):
    q = tth_2_q(tth, wavelength=wavelength)
    q[np.isnan(q)] = 0
    q_range = [np.min(q), np.max(q)]

    d = d_spacings(q_range, a, b, c, alpha, beta, gamma)
    q_cen = 2 * np.pi / np.array(d)

    args = []
    for i in range(len(q_cen)):
        cen_idx = np.argmin(np.abs(q - q_cen[i]))
        amp = intensity[cen_idx]
        q_vals = q[cen_idx - window : cen_idx + window]
        int_vals = intensity[cen_idx - window : cen_idx + window]
        std_tth = np.sqrt(np.cov(q_vals, aweights=int_vals))
        fwhm = std_tth * 2 * np.sqrt(2 * np.log(2))

        args.extend([
            amp,
            fwhm
        ]) 
    
    return args


def pattern_estimate(tth, *args, wavelength=test.wavelength):
    a, b, c, alpha, beta, gamma = args[:6]
    int_fwhm = args[6:]
    
    q = tth_2_q(tth, wavelength=wavelength)
    q[np.isnan(q)] = 0
    q_range = [np.min(q), np.max(q)]

    d = d_spacings(q_range, a, b, c, alpha, beta, gamma)
    q_cen = 2 * np.pi / np.array(d)

    args = [0]
    for i in range(len(q_cen)):
        args.extend([
            int_fwhm[::2][i],
            q_cen[i],
            int_fwhm[1::2][i]
        ])

    intensity = LorentzianFunctions.multi_1d(q, *args)

    return intensity


def pattern_estimate_from_phase_guess(tth, intensity, window=20):
    phases = [test.phases['304SS-austenite'],
              test.phases['platinum'],
              test.phases['tungsten_2']]

    q = tth_2_q(tth, wavelength=test.wavelength)
    q[np.isnan(q)] = 0

    d = []
    for phase in phases:
        lattice_params = list(phase.lattice._parameters.values())
        lattice_params[3:] = np.radians(lattice_params[3:])

        d.extend(hkl_2_d(phase.reflections['hkl'],
                         *lattice_params))

    q_cen = 2 * np.pi / np.array(d)

    args = [np.min(intensity[20:-20])]
    for i in range(len(q_cen)):
        cen_idx = np.argmin(np.abs(q - q_cen[i]))
        amp = intensity[cen_idx] - np.min(intensity)
        
        q_vals = q[cen_idx - window : cen_idx + window]
        int_vals = intensity[cen_idx - window : cen_idx + window]
        int_vals[int_vals < 0] = 0
        
        std_tth = np.sqrt(np.cov(q_vals, aweights=int_vals))
        fwhm = std_tth * 2 * np.sqrt(2 * np.log(2))

        args.extend([
            amp,
            fwhm
        ])

    return args


def pattern_estimate_from_phase(tth, *args):
    phases = [test.phases['304SS-austenite'],
              test.phases['platinum'],
              test.phases['tungsten_2']]
    lattice_params = np.array(args[:6 * len(phases)]).reshape(-1, 6)
    offset = args[6 * len(phases)]
    int_fwhm = args[6 * len(phases) + 1:]

    q = tth_2_q(tth, wavelength=test.wavelength)
    q[np.isnan(q)] = 0

    d = []
    for i, phase in enumerate(phases):
        d.extend(hkl_2_d(phase.reflections['hkl'],
                         *lattice_params[i]))
    
    q_cen = 2 * np.pi / np.array(d)

    args = [offset]
    for i in range(len(q_cen)):
        args.extend([
            int_fwhm[::2][i],
            q_cen[i],
            int_fwhm[1::2][i]
        ])

    intensity = LorentzianFunctions.multi_1d(q, *args)

    return intensity

def generate_bounds(phases, args):

    lattice_params = np.array(args[:6 * len(phases)]).reshape(-1, 6)
    offset = args[6 * len(phases)]
    int_fwhm = args[6 * len(phases) + 1:]

    low_bounds, upr_bounds = [], []

    for lattice_params_i in lattice_params:
        a, b, c, alpha, beta, gamma = lattice_params_i

        for const in [a, b, c]:
            # Ridiculous bounds of 5% strain
            low_bounds.append(0.95 * const)
            upr_bounds.append(1.05 * const)
        
        for angle in [alpha, beta, gamma]:
            low_bounds.append(angle - np.radians(5))
            upr_bounds.append(angle + np.radians(5))

    # offset bound
    low_bounds.append(0)
    upr_bounds.append(5 * offset)

    # amp and fwhm
    for i, val in enumerate(int_fwhm):
        if i % 2 == 0: # even, amp
            low_bounds.append(0)
            upr_bounds.append(3 * val)
        else: # odd, fwhm
            low_bounds.append(0.001)
            upr_bounds.append(3 * val)

    return [low_bounds, upr_bounds]


In [ ]:
test.map_extent()

[-10.203479, 9.687759, -20.038216, 20.00252]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

xrf_cen = 2700
x_signal = rescale_array(np.sum(xrfmap[:, :, xrf_cen - 15:xrf_cen + 15], axis=(-1)))
x_energy = np.mean(xrf_energy[xrf_cen - 15:xrf_cen + 15])

tth_cen = 830
y_signal = rescale_array(np.sum(test.map.integrations[:, :, tth_cen - 30:tth_cen + 30], axis=(-1)))
y_tth = np.mean(test.tth[tth_cen - 30:tth_cen + 30])

ax.scatter(x_signal, y_signal, s=1, alpha=0.5, c='k')
ax.set_xlabel(f'XRF Energy around {x_energy:.2f} keV')
ax.set_ylabel(f'XRD 2θ around {y_tth:.2f}°')

corr_signal = x_signal * y_signal

fig.show()

test.plot_map(corr_signal)

In [ ]:
xrf_dict.keys()

dict_keys(['S_K', 'Ga_K', 'Si_K', 'Sb_L', 'Ni_K', 'Fe_K', 'Ar_K', 'W_L', 'Pt_L', 'compton', 'elastic', 'snip_bkg', 'r_factor', 'sel_cnt', 'total_cnt', 'i0'])

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array

xrf_stack = []

for key in xrf_dict.keys():
    if key in ['snip_bkg', 'r_factor', 'sel_cnt', 'total_cnt', 'i0']:
        continue

    xrf_stack.append(rescale_array(xrf_dict[key].copy()))

xrf_stack = np.stack(xrf_stack)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(test.tth, np.max(test.map.integrations, axis=(0, 1)))

fig.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

#x, bins, _ = ax.hist(test.spots['guess_cen_tth'].values, bins=2000, weights=test.spots['guess_height'].values, label='spot center', density=True)
x, bins, _ = ax.hist(test.spots['guess_cen_tth'].values, bins=2000, label='spot center', density=True)
ax.plot(test.tth, rescale_array(np.max(test.map.integrations, axis=(0, 1)), lower=np.min(x), upper=np.max(x)), label='max intensity')

ax.legend()

plt.show()

In [ ]:
test.integrate1d_map()

Integrated images to 1D...

100%|██████████| 3321/3321 [01:18<00:00, 42.25it/s]


In [ ]:
from skimage.feature import peak_local_max
from scipy.signal import find_peaks

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

plot_int = rescale_array(np.max(test.map.integrations, axis=(0, 1)))
#peaks = peak_local_max(plot_int,
#                       threshold_rel=0.05)

peaks, _ = find_peaks(plot_int, prominence=0.01, distance=4)

ax.scatter(test.tth[peaks], plot_int[peaks], s=2, c='r')
ax.plot(test.tth, plot_int)


fig.show()

In [ ]:
peak_maps = np.empty((*test.map.map_shape, len(peaks)))

params = list(popt.copy())
params.pop(0)
for i in range(len(peaks)):
    peak_map = np.sum(test.map.integrations * SpotModel.func_1d(test.tth, *params[:3]), axis=(-1))
    peak_maps[:, :, i] = rescale_array(peak_map)
    del params[:3]

peak_maps = peak_maps.T

In [ ]:
features = np.vstack([xrf_stack, peak_maps])
features = features.reshape(features.shape[0], -1)

In [ ]:
features.shape

(56, 3321)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

x_signal = features[0]
y_signal = features[3]

ax.scatter(x_signal, y_signal, s=5, alpha=0.5, c='k')

fig.show()

In [ ]:
projections = features[:, np.newaxis] * features[np.newaxis, :]

In [ ]:
correlations = np.empty((features.shape[0], features.shape[0]))

for index in range(np.prod(correlations.shape)):
    indices = np.unravel_index(index, correlations.shape)

    correlation = np.dot(features[indices[0]], features[indices[1]]) / (np.linalg.norm(features[indices[0]]) * np.linalg.norm(features[indices[1]]))
    correlations[indices] = correlation

In [ ]:
projections.shape

(56, 56, 3321)

In [ ]:
correlations.shape

(56, 56)

In [ ]:
pairs = []

for index in range(np.prod(correlations.shape)):
    indices = np.unravel_index(index, correlations.shape)

    if (correlations[indices] > 0.95
        and indices[0] != indices[1]
        and indices[::-1] not in pairs):
        pairs.append(indices)

In [ ]:
pseudo_phases = []
assigned_features = []

for i, pair in enumerate(pairs):

    NEW_pseudo_PHASE = True
    for pseudo_phase in pseudo_phases:
        if (pair[0] in pseudo_phase
            or pair[1] in pseudo_phase):
            pseudo_phase.extend(list(pair))
            NEW_pseudo_PHASE = False
            continue

    for pseudo_phase in pseudo_phases:
        if (pair[0] in pseudo_phase
            and pair[1] in pseudo_phase):
            pseudo_phase.extend(list(pair))
            NEW_pseudo_PHASE = False
            continue
    
    if NEW_pseudo_PHASE:
        pseudo_phases.append(list(pair))

    #if i == 55:
    #    break

# Reduce to unique features
#for i, pseudo_phase in enumerate(pseudo_phases):
#    pseudo_phases[i] = list(np.unique(np.array(pseudo_phase)))

# Combined spearated correlated psued0_phases
for i, pseudo_phase in enumerate(pseudo_phases):
    for feature_i in pseudo_phase:
        matches = [feature_i in x for x in pseudo_phases]
        if np.sum(matches) > 1:
            combined_phase_index = list(np.where(matches)[0])[1] # zero index is the same, others will be combined in next pass
            pseudo_phases[i].extend(pseudo_phases[combined_phase_index])
            pseudo_phases.pop(combined_phase_index)

# Reduce to unique features
for i, pseudo_phase in enumerate(pseudo_phases):
    pseudo_phases[i] = list(np.unique(np.array(pseudo_phase)))


In [ ]:

pseudo_phases = []
for pair in pairs:

    f0 = pair[0]
    f1 = pair[1]

    f0_matches = [np.array(pair_i)[pair_i != f0][0] for pair_i in pairs if f0 in pair_i]
    f1_matches = [np.array(pair_i)[pair_i != f1][0] for pair_i in pairs if f1 in pair_i]

    f3_matches = [f3 for f3 in f0_matches if f3 in f1_matches]

    if len(f3_matches) > 0:
        pseudo_phase = [*pair, *f3_matches]
        pseudo_phases.append(pseudo_phase)



In [ ]:
# Reduce to unique features
for i, pseudo_phase in enumerate(pseudo_phases):
    pseudo_phases[i] = list(np.unique(np.array(pseudo_phase)))

In [ ]:
plot_map = np.zeros_like(features[0])

for i in pseudo_phases[-1]:
    plot_map += features[i]

test.plot_map(plot_map.reshape(test.map.map_shape[::-1]).T)

In [ ]:
ext = np.max(np.abs([np.min(correlations), np.max(correlations)]))

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(correlations, vmin=-ext, vmax=ext, cmap='bwr')
fig.colorbar(im, ax=ax)

fig.show()

In [ ]:
ext = np.max(np.abs([np.min(correlations), np.max(correlations)]))

fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(np.max(projections, axis=-1))
fig.colorbar(im, ax=ax)

fig.show()

In [ ]:
xrf_dict.keys()

dict_keys(['S_K', 'Ga_K', 'Si_K', 'Sb_L', 'Ni_K', 'Fe_K', 'Ar_K', 'W_L', 'Pt_L', 'compton', 'elastic', 'snip_bkg', 'r_factor', 'sel_cnt', 'total_cnt', 'i0'])

In [ ]:
from xrdmaptools.plot.interactive_plotting import interactive_dynamic_2d_plot

interactive_dynamic_2d_plot(np.transpose(projections.reshape(61, 61, *test.map.map_shape[::-1]), (0, 1, 3, 2)),
                                display_map=np.max(projections, axis=-1), display_title=None,
                                map_vmin=None, map_vmax=None,
                                cmap='viridis', marker_color='red')

(81, 41, 3086)

In [ ]:
from skimage.transform import resize

X = np.vstack([rescale_array(resize(xrfmap.copy().T[:2048], (1024, 41, 81))), rescale_array(resize(test.map.integrations.copy().T, (1000, 41, 81)))])
X = X.astype(np.float16)

In [ ]:
raw_projections = X[:, np.newaxis] * X[np.newaxis, :]

In [ ]:
raw_projections

(2024, 2024, 41, 81)

In [ ]:
max_projections = np.max(raw_projections, axis=(2, 3))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(np.max(raw_projections, axis=(2, 3)))
fig.colorbar(im, ax=ax)

fig.show()

In [ ]:
from xrdmaptools.utilities.utilities import label_nearest_spots

out = label_nearest_spots(features.T, max_dist=1)

In [ ]:
np.unique(out[:, -1])

array([1., 2.])

In [ ]:
out.shape

(3321, 57)

In [ ]:
test.plot_map(out[:, -1].reshape(test.map.map_shape[::-1]).T, cmap='tab20')

In [ ]:
from sklearn.manifold import TSNE, Isomap

tsne = TSNE(n_components=3)
out = tsne.fit_transform(features.T)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.scatter(out[:, 0], out[:, 1], s=1, c='k')

fig.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200, subplot_kw={'projection':'3d'})

ax.scatter(out[:, 0], out[:, 1], out[:, 2], s=1, c='k')

fig.show()

In [ ]:
xrf_dict.keys()

dict_keys(['S_K', 'Ga_K', 'Si_K', 'Sb_L', 'Ni_K', 'Fe_K', 'Ar_K', 'W_L', 'Pt_L', 'compton', 'elastic', 'snip_bkg', 'r_factor', 'sel_cnt', 'total_cnt', 'i0'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.scatter(out[:, 0], out[:, 1], s=1, c=features[7])

fig.show()

In [ ]:
X = np.vstack([rescale_array(xrfmap.copy().T, arr_min=0, upper=100), test.map.integrations.copy().T])
X = X.reshape(-1, test.map.num_images)

In [ ]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=15).fit(features.T)
#kmeans = KMeans(n_clusters=4).fit(out)
#kmeans = KMeans(n_clusters=6).fit(X.T)

In [ ]:
from sklearn.cluster import SpectralClustering

kmeans = SpectralClustering(n_clusters=6).fit(features.T)

In [ ]:
test.plot_map(kmeans.labels_.reshape(test.map.map_shape[::-1]).T, cmap='tab20')

In [ ]:
features

(56, 3321)

In [ ]:
from sklearn.decomposition import PCA

n_components = 24

X = features.reshape(-1, *test.map.map_shape[::-1])

h, w = X.shape[1], X.shape[2]

print("Extracting the top %d eigenpatterns from %d patterns"
      % (n_components, X.shape[0]))
t0 = ttime.time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X.reshape(len(X), h * w))

x_pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit_transform(X.reshape(len(X), h * w))

eigen = pca.components_.reshape((n_components, h, w))

print("done in %0.3fs" % (ttime.time() - t0))

Extracting the top 24 eigenpatterns from 56 patterns
done in 0.040s


In [ ]:
from sklearn.decomposition import NMF

n_components = 6

h, w = X.shape[1], X.shape[2]

print("Extracting the top %d eigenpatterns from %d patterns"
      % (n_components, X.shape[0]))
t0 = ttime.time()
pca = NMF(n_components=n_components).fit(X.reshape(len(X), h * w))

x_pca = NMF(n_components=n_components).fit_transform(X.reshape(len(X), h * w))

eigen = pca.components_.reshape((n_components, h, w))

print("done in %0.3fs" % (ttime.time() - t0))

Extracting the top 6 eigenpatterns from 56 patterns
done in 0.233s


c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\sklearn\decomposition\_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\sklearn\decomposition\_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
eigen.shape

(12, 41, 81)

In [ ]:
fig, ax = plt.subplots(6, 1, figsize=(5, 5), dpi=200)

for i, axi in enumerate(ax):
    axi.imshow(eigen[i])
    axi.set_xticks([])
    axi.set_yticks([])

fig.show()

In [ ]:
blob_image = test_image * out[1]
tth, xrd_blob = test.integrate1d_image(blob_image)

In [ ]:
# bad azimuthal integration for testing...

tth_step = np.mean(np.diff(test.tth))

tth_arr = test.tth_arr

xrd_int = []
xrd_count = []
xrd_mean = []

for tth_i in test.tth:    
    tth_mask = np.all([tth_arr > tth_i - tth_step / 2,
                       tth_arr < tth_i + tth_step / 2], axis=0)
    
    xrd_num = np.sum(tth_mask)
    if xrd_num > 0:
        xrd_sum = np.max(test_image[tth_mask])
    else:
        xrd_sum = 0
    

    xrd_int.append(xrd_sum)
    xrd_count.append(xrd_num)
    xrd_mean.append(xrd_sum / xrd_num)

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array

phase = test.phases['stibnite']

#amp, x0, fwhm
p0 = [0]
fwhm = 0.1

for i in range(len(phase.reflections['hkl'])):
    p0.append(phase.reflections['int'][i]) # amp
    #p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(fwhm) #fwhm

print(p0)
calc_xrd = GaussianFunctions.multi_1d(test.tth, *p0)
calc_xrd = rescale_array(calc_xrd, 0, 1)

p0 = [0]

for i in range(len(phase.reflections['hkl'])):
    #p0.append(phase.reflections['int'][i]) # amp
    p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(fwhm) #fwhm

print(p0)
xrd_mask = GaussianFunctions.multi_1d(test.tth, *p0)
xrd_mask = rescale_array(xrd_mask, 0, 1)

[0, 9.229291336271558, 9.79391082638766, 0.1, 36.35607901077193, 9.947146203273547, 0.1, 4.538280049707817, 10.003918314667498, 0.1, 100.0, 10.287225195166958, 0.1, 52.062489968299765, 11.203251751192864, 0.1, 1.658225681571489, 11.362641176238784, 0.1, 48.06295385493908, 11.393796968970216, 0.1, 9.900411012398704, 11.643708106935721, 0.1, 5.248101535050058, 11.661966917135357, 0.1, 1.1602521706592916, 12.582758827750855, 0.1, 2.2555150424646198, 12.672729437925394, 0.1, 25.048460183122817, 12.876186484052784, 0.1, 4.234661867419972, 12.977123761304796, 0.1, 52.24625259837442, 13.05898401587513, 0.1, 33.33036447295974, 13.28339128375106, 0.1, 1.5935045076294014, 13.397236314228078, 0.1, 41.25420645687469, 13.610946139998532, 0.1, 14.705669354514475, 13.652674016230323, 0.1, 23.458776258920437, 14.095268845670564, 0.1, 0.6725457075094272, 14.155715299043614, 0.1, 17.725053635792502, 14.681782907483074, 0.1, 35.33326441299243, 14.706014883021318, 0.1, 1.7257650652317706, 15.6492826713532

In [ ]:
integrations = plot_integrations.copy()
integrations = test.map.integrations.copy()
#integrations = integration_map.copy()

#rescale_array(integrations, arr_min = 0, upper=1)

proj_map = np.dot(integrations, calc_xrd)
proj_mask = np.dot(integrations, xrd_mask)

norms = np.linalg.norm(integrations, axis=-1)
norm_integrations = (integrations.T / norms.T).T

norm_calc_xrd = calc_xrd / np.linalg.norm(calc_xrd)
norm_xrd_mask = xrd_mask / np.linalg.norm(xrd_mask)

corr_map = np.dot(norm_integrations, norm_calc_xrd)
corr_mask = np.dot(norm_integrations, norm_xrd_mask)

window_int = integrations * xrd_mask
window_int /= np.linalg.norm(window_int)

double_map = np.dot(window_int, norm_calc_xrd)
double_mask = np.dot(window_int, norm_xrd_mask)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(7, 5), dpi=200)
axes = ax.ravel()

maps = [
        proj_map, corr_map, double_map,
        #proj_mask, corr_mask, double_mask
        ]
titles = [
          'Powder Projection',
          'Powder Correlation',
          'Powder Window Correlation',
         # 'Planes Projection',
         # 'Planes Correlation',
         # 'Planes Window Correlation'
          ]

fig.suptitle(f'{phase.name} mapping')


for axi, mapi, title in zip(axes, maps, titles):
    im = axi.imshow(mapi)
    fig.colorbar(im, ax=axi, shrink=0.7)
    axi.set_title(title)

fig.show()

In [ ]:
plt.close('all')

In [ ]:
from xrdmaptools.plot.interactive_plotting import interactive_dynamic_1d_plot

interactive_dynamic_1d_plot(rescale_array(integration_map, arr_min=0, upper=100), tth=test.tth, 
                                bkg_removal=None, ball_size=None, normalize=None,
                                display_map=None, display_title=None,
                                map_vmin=None, map_vmax=None,
                                cmap='viridis', marker_color='red')

In [ ]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
#test.load_phase('Barker\\1521772COD.cif', wd=cif_dir, phase_name="liNbO3 old")
#test.load_phase('Barker\\LiNbO3.cif', wd=cif_dir, phase_name="LiNbO3 148")
#test.load_phase('Unknown\\LiNbO3.cif', wd=cif_dir, phase_name="LiNbO3 new")
test.load_phase('AMCSD\\Platinum_0011157.cif', wd=cif_dir, phase_name="platinum")
#test.load_phase('AMCSD\\Calcite_0000984.cif', wd=cif_dir, phase_name="calcite")
test.load_phase('AMCSD\\Stibnite_0008636.cif', wd=cif_dir, phase_name="stibnite")
#test.load_phase('AMCSD\\Silicon_0011243.cif', wd=cif_dir, phase_name="silicon")
#test.load_phase('AMCSD\\Corundum_0009327.cif', wd=cif_dir, phase_name="sapphire")
#test.load_phase('AMCSD\\Hematite_0017806.cif', wd=cif_dir, phase_name="hematite")
#test.load_phase('AMCSD\\Iron-alpha_0011214.cif', wd=cif_dir, phase_name="iron-ferrite")
#test.load_phase('AMCSD\\Iron-beta_0011215.cif', wd=cif_dir, phase_name="iron-beta")
#test.load_phase('AMCSD\\Iron-delta_0011216.cif', wd=cif_dir, phase_name="iron-delta")
#test.load_phase('AMCSD\\Iron_0011146.cif', wd=cif_dir, phase_name="iron-austenite")
#test.load_phase('AMCSD\\Cementite_0013523.cif', wd=cif_dir, phase_name="cementite")
#test.load_phase('AMCSD\\Graphite_0011247.cif', wd=cif_dir, phase_name="graphite")
#test.load_phase('AMCSD\\Tungsten_0011261.cif', wd=cif_dir, phase_name="tungsten_1")
test.load_phase('AMCSD\\Tungsten_0011236.cif', wd=cif_dir, phase_name="tungsten_2")
test.load_phase('AMCSD\\Iron_0011146.cif', wd=cif_dir, phase_name="austenite")
test.phases['austenite'].a = 3.529

In [ ]:
[phase.get_hkl_reflections(tth_range=(np.min(test.tth), np.max(test.tth)),
                            ignore_less=0)
    for phase in test.phases.values()];

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array

phase = test.phases['stibnite']

#amp, x0, fwhm
p0 = [0]

for i in range(len(phase.reflections['hkl'])):
    p0.append(phase.reflections['int'][i]) # amp
    #p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(0.1) #fwhm

calc_xrd = GaussianFunctions.multi_1d(test.tth, *p0)
calc_xrd = rescale_array(calc_xrd, 0, 1)

p0 = [0]

for i in range(len(phase.reflections['hkl'])):
    #p0.append(phase.reflections['int'][i]) # amp
    p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(0.1) #fwhm

xrd_mask = GaussianFunctions.multi_1d(test.tth, *p0)
xrd_mask = rescale_array(calc_xrd, 0, 1)

In [ ]:
integrations = integration_map
#integrations = test.map.integrations

proj_map = np.dot(integrations, calc_xrd)

norms = np.linalg.norm(integrations, axis=-1)
norm_integrations = (integrations.T / norms.T).T

norm_calc_xrd = calc_xrd / np.linalg.norm(calc_xrd)

corr_map = np.dot(norm_integrations, norm_calc_xrd)

window_int = integrations * xrd_mask
window_int /= np.linalg.norm(window_int)

double_map = np.dot(window_int, norm_calc_xrd)

In [ ]:
test.plot_map(proj_map)

In [ ]:
test.plot_map(corr_map)

In [ ]:
test.plot_map(double_map)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(test.tth, rescale_array(calc_xrd, arr_min=0, upper=1))
ax.plot(test.tth, test.map.integrations[8, 160])
ax.plot(test.tth, rescale_array(calc_xrd, arr_min=0, upper=1) * rescale_array(test.map.integrations[8, 160], arr_min=0, upper=1))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(tth, calc_xrd)

fig.show()

In [ ]:
pixel_indices = [0, 0] #[26, 21]
skip = 500

#fig, ax = plt.subplots(1, 1, figsize=(5, 10), dpi=200, subplot_kw={'projection':'3d'})
fig = plt.figure(figsize=(10, 5), dpi=200)
ax = fig.add_axes([0, 0, 0.5, 1], projection='3d')
#ax = fig.add_subplot(projection='3d')

plot_qs = euler_rotation(all_qs, 10, -20, 0)
recip_latt = ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
ax.scatter(0, 0, 0, c='k', s=10)

# Plot full Ewald sphere
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
radius = 2 * np.pi / test.wavelength
x =  radius * np.outer(np.cos(u), np.sin(v))
y = radius * np.outer(np.sin(u), np.sin(v))
z = radius * np.outer(np.ones(np.size(u)), np.cos(v))
#ax.plot_surface(x, y, z - radius, alpha=0.2, color='k', label='Ewald sphere')

q = get_q_vect(test.tth_arr, test.chi_arr, wavelength=test.wavelength)

if pixel_indices is not None:
    pixel_df = test.spots[(test.spots['map_x'] == pixel_indices[0])
                            & (test.spots['map_y'] == pixel_indices[1])].copy()


if pixel_indices is not None:
    ax.scatter(*pixel_df[['qx', 'qy', 'qz']].values.T, s=1, c='r', label='spots')

# Sample geometry
ax.quiver([0, 0], [0, 0], [-2 * radius, -radius], [0, 0], [0, 0], [radius, radius], colors='k')
ax.scatter(0, 0, 0, marker='o', s=10, facecolors='none', edgecolors='k', label='transmission')
ax.scatter(0, 0, -radius, marker='h', s=10, c='b', label='sample')

# Plot sampled Ewald sphere
q_mask = q[:, test.map.mask]
ax.plot_trisurf(q_mask[0].ravel()[::skip],
                q_mask[1].ravel()[::skip],
                q_mask[2].ravel()[::skip],
                alpha=0.5, label='detector')

phi1, PHI, phi2 = 10, -20, 0
euler_angles = {'phi1' : phi1,
                'PHI' : PHI,
                'phi2': phi2}
euler_bounds = [[-180, 180], [0, 180], [-180, 180]]
slider_lst = []
update_lst = []

slider_vpos = np.linspace(0.8, 0.1, 3)

for i, key in enumerate(euler_angles.keys()):
    slider_ax = fig.add_axes

    # Make a horizontal slider to control the frequency.
    axfreq = fig.add_axes([0.7, slider_vpos[i], 0.2, 0.03])
    euler_slider = Slider(
        ax=axfreq,
        label=f'{key} [deg]',
        valmin=euler_bounds[i][0],
        valmax=euler_bounds[i][1],
        valinit=euler_angles[key],
    )

    slider_lst.append(euler_slider)

    # The function to be called anytime a slider's value changes
    def update_factory(key):
        def update(val):
            global recip_latt
            euler_angles[key] = val
            plot_qs = euler_rotation(all_qs, *euler_angles.values())
            recip_latt.remove()
            recip_latt = ax.scatter(*np.asarray(plot_qs).T, c='k', s=all_fs / 50, alpha=0.75)
            fig.canvas.draw_idle()
        return update

    update_lst.append(update_factory(key))
    slider_lst[i].on_changed(update_lst[i])

ax.set_xlabel('qx [Å⁻¹]')
ax.set_ylabel('qy [Å⁻¹]')
ax.set_zlabel('qz [Å⁻¹]')
ax.set_aspect('equal')

plt.show()